# Entorno de trabajo: Crear un dataset de noticias de prensa a partir del motor de búsqueda Sophia

Este notebook muestra una manera de extraer noticias desde el motor de búsqueda de _Sophia_, a partir de una **palabra clave**, un **país** y un **periodo de tiempo**.


_Sophia_ es un proyecto I+i+D desarrollado en la Universidad Austral de Chile desde 2019. El motor de búsqueda se encuentra en fase de mejoramiento continuo. Por cualquier pregunta o problema técnica con este notebook, no dudar en contactarme por Discord (@MatthieuVernier) o por correo mvernier@inf.uach.cl.

## 0. Librerías útiles

- Elasticsearch es el motor de búsqueda utilizado por Sophia
- Requiere la instalación del paquete elasticsearch

In [2]:
!python -V

Python 3.8.10


In [3]:
!pip install elasticsearch

In [4]:
import elasticsearch
from datetime import datetime

In [5]:
elasticsearch.__version__

(7, 16, 3)

## 1. Parámetros de la búsqueda

Características del motor de búsqueda:  
- El formato de las fechas es: YYYY-MM-DD
- El motor de búsqueda incluye noticias por distintos paises, pero se recomienda utilizar "chile" por el momento.
- El motor de búsqueda incluye datos entre el año 2000 y el año 2021.

In [27]:
country="chile"
from_="2010-01-01"
to_="2021-12-31"
media_outlets=["elllanquihue","elaustral","laestrelladechiloe","elheraldoaustral","radiosago",
               "elrepuertero","elvacanudo","elhuemul","seminariolocal","elquellonino","elinsular",
               "radiopudeto","radioacogida","elcalbucano","segundos33",
               "prensadelestuario","fresiaahora","soychiloe"]

keyword="cambio climático"
simple_keyword=False

## 2. Parámetros de conexión al motor de búsqueda de Sophia

In [28]:
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS= "dummy_elastic"

In [29]:
#NO MODIFICAR

es = elasticsearch.Elasticsearch(
    IP,
    #port=PORT,
    http_auth=(USER, PASS)
)

## 3. Búsqueda de noticias de prensa

- Preparación de la consulta en el formato solicitado por Elasticsearch

In [41]:
#NO MODIFICAR
match=""
if (simple_keyword):
    match="match"
else:
    match="match_phrase"

query = { 
    "bool": { 
      "must": [
        {match: { "text":keyword}}

      ],
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }}},
        { "term":  { "country": country }},
        { "terms":  { "media_outlet": media_outlets }} 
    ]
    }  
}

- Envio de la consulta
- No se puede devolver más de 10.000 noticias. En caso de obtener 10.000 respuestas, se recomienda buscar en un periodo de tiempo más corto o modificar la consulta.

In [42]:
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 2686 noticias encontradas...


## 4. Guardar los datos en un dataframe Pandas y en un archivo CSV

In [43]:
import pandas as pd

data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[],'search':[]}

df = pd.DataFrame(data)  
  
for hit in res['hits']['hits']:
    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    search = keyword
    
    new_row = {'id_news':id_news, 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date, 'search':search}
    
    df = df.append(new_row, ignore_index=True)

df

,id_news,country,media_outlet,url,title,text,date,search
0,22581387.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2020/02/02/...,Encuesta revela qué pensamos sobre el cambio c...,El 93% de los latinoamericanos está de acuerdo...,2020-02-02,cambio climático
1,21425188.0,chile,elhuemul,https://www.elhuemul.cl/2015/11/30/futaleufu-s...,Futaleufú se adhiere a la marcha mundial contr...,Entre las actividades se desarrolló una marcha...,2015-11-30,cambio climático
2,22387263.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2019/1...,Cambio climático: qué piensan y sienten en tod...,El 93% de los latinoamericanos está de acuerdo...,2019-12-30,cambio climático
3,22580118.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2019/1...,Cambio climático: qué piensan y sienten en tod...,El 93% de los latinoamericanos está de acuerdo...,2019-12-30,cambio climático
4,21426965.0,chile,elhuemul,https://www.elhuemul.cl/2015/11/17/el-29-de-no...,El 29 de noviembre Futaleufu se sumará a la Ma...,"La iniciativa, propulsada en Futaleufu por el ...",2015-11-17,cambio climático
...,...,...,...,...,...,...,...,...
2681,21935932.0,chile,radiosago,https://www.radiosago.cl/kast-dice-que-chile-m...,Kast dice que Chile «merece» un Presidente que...,Una cargada agenda ha tenido en los últimos dí...,2021-12-12,cambio climático
2682,21451085.0,chile,elrepuertero,https://www.elrepuertero.cl/noticia/politica/l...,Los ejes que marcaron la Cuenta Pública 2020: ...,"Este viernes, a partir de las 20:00 horas, el ...",2020-08-01,cambio climático
2683,21456843.0,chile,elvacanudo,https://www.elvacanudo.cl/noticia/politica/los...,Los ejes que marcaron la Cuenta Pública 2020: ...,"Este viernes, a partir de las 20:00 horas, el ...",2020-08-01,cambio climático
2684,21410426.0,chile,radiopudeto,http://www.pudeto.cl/?p=43808,GREENPEACE REVELA EVIDENCIA CIENTÍFICA QUE IND...,Greenpeace dio a conocer un informe respecto d...,2016-09-05,cambio climático


- Verifiquemos si existen noticias con la misma URL

In [44]:
df['url'].duplicated().any()

True

- Eliminar las noticias duplicadas

In [45]:
df = df.drop_duplicates(subset='url', keep='first')

In [46]:
df

,id_news,country,media_outlet,url,title,text,date,search
0,22581387.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2020/02/02/...,Encuesta revela qué pensamos sobre el cambio c...,El 93% de los latinoamericanos está de acuerdo...,2020-02-02,cambio climático
1,21425188.0,chile,elhuemul,https://www.elhuemul.cl/2015/11/30/futaleufu-s...,Futaleufú se adhiere a la marcha mundial contr...,Entre las actividades se desarrolló una marcha...,2015-11-30,cambio climático
2,22387263.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2019/1...,Cambio climático: qué piensan y sienten en tod...,El 93% de los latinoamericanos está de acuerdo...,2019-12-30,cambio climático
4,21426965.0,chile,elhuemul,https://www.elhuemul.cl/2015/11/17/el-29-de-no...,El 29 de noviembre Futaleufu se sumará a la Ma...,"La iniciativa, propulsada en Futaleufu por el ...",2015-11-17,cambio climático
5,21398696.0,chile,elhuemul,https://www.elhuemul.cl/2019/11/18/comunas-de-...,Comunas de Palena y Chaitén preparan seminario...,El cambio climático es el mal de nuestro tiemp...,2019-11-18,cambio climático
...,...,...,...,...,...,...,...,...
2681,21935932.0,chile,radiosago,https://www.radiosago.cl/kast-dice-que-chile-m...,Kast dice que Chile «merece» un Presidente que...,Una cargada agenda ha tenido en los últimos dí...,2021-12-12,cambio climático
2682,21451085.0,chile,elrepuertero,https://www.elrepuertero.cl/noticia/politica/l...,Los ejes que marcaron la Cuenta Pública 2020: ...,"Este viernes, a partir de las 20:00 horas, el ...",2020-08-01,cambio climático
2683,21456843.0,chile,elvacanudo,https://www.elvacanudo.cl/noticia/politica/los...,Los ejes que marcaron la Cuenta Pública 2020: ...,"Este viernes, a partir de las 20:00 horas, el ...",2020-08-01,cambio climático
2684,21410426.0,chile,radiopudeto,http://www.pudeto.cl/?p=43808,GREENPEACE REVELA EVIDENCIA CIENTÍFICA QUE IND...,Greenpeace dio a conocer un informe respecto d...,2016-09-05,cambio climático


- Guardar los archivos en un CSV

In [47]:
nombre_archivo=keyword.replace(' ', '')+"_"+country+"_"+from_+"_"+to_+".csv"
print("El archivo se llama:")
print(nombre_archivo)
df.to_csv("./"+nombre_archivo)

El archivo se llama:
cambioclimático_chile_2010-01-01_2021-12-31.csv


- Realizar algunas consultas SQL sobre el dataset

In [48]:
from pandasql import sqldf 

In [49]:
result = sqldf("SELECT media_outlet,count(*) FROM df GROUP BY media_outlet ORDER BY count(*) DESC")
result

,media_outlet,count(*)
0,elllanquihue,465
1,elaustral,386
2,laestrelladechiloe,357
3,elvacanudo,236
4,elrepuertero,230
5,elheraldoaustral,217
6,radiosago,142
7,elhuemul,90
8,radiopudeto,42
9,seminariolocal,19
